In [ ]:
from Crypto.Hash import MD5
from sympy import randprime
import ctypes

In [ ]:
#Nadawca (p,q) - para kluczy randprime N>=P*Q, Fi = (P-1)(Q-1), e=65537, D=odwrotnością modularną w arytmetyce Fi d=pow(e,-1,Fi),
#Ma do przesłania wiadomość tekstową 'w', wiadomość nie będzie obchodzić, żeby była tajna
#Chcemy dodstać podpis wiadomości
#Zdefiniować hash z wiadomości MD5 z wiadomości, 128 bitowy hash dla dowolnej wiadomości
#hash podnosimy do potęgi d, modulo N (deszyfrowanie hashu - tak jakby)

#Odbiorca (e, N)
#Podniesie sobie ten szyfr hashu e modulo N
#Może też policzyć sobie hash

In [ ]:
P = randprime(pow(2,64), pow(2,65))
Q = randprime(pow(2,64), pow(2,65))
N = P*Q
Fi = (P-1)*(Q-1)
E = 65537
D = pow(E, -1, Fi)
W = "Widzew"
W_bytes = W.encode()
_hash = MD5.new()
W_uint8_arr = (ctypes.c_uint8 * len(W_bytes)).from_buffer_copy(W_bytes)
_hash.update(W_bytes)
W_md5_bytes = _hash.digest()
W_md5_hex = _hash.hexdigest()

print(W_md5_bytes)
print(W_md5_hex)

In [ ]:
W_md5_int = int.from_bytes(W_md5_bytes, byteorder='big')
signature = pow(W_md5_int, D, N)

signed_message = f"{signature}{W_md5_int}"
signed_message_bytes = signed_message.encode()

print(W_md5_int)
print(signature)
print(signed_message_bytes)

In [ ]:
def sign_message(message, D, N):
    # Calculate the hash of the message
    message_hash = int(hashlib.md5(message.encode()).hexdigest(), 16)

    # Generate the signature
    signature = pow(message_hash, D, N)

    # Convert signature to bytes
    signature_bytes = signature.to_bytes((signature.bit_length() + 7) // 8, 'big')

    # Convert message to bytes
    message_bytes = message.encode()

    # Combine the signature and message bytes
    combined_message = signature_bytes + message_bytes

    return combined_message

In [ ]:
def encrypt(message):
    W_bytes = message.encode()
    _hash = MD5.new()
    W_uint8_arr = (ctypes.c_uint8 * len(W_bytes)).from_buffer_copy(W_bytes)
    _hash.update(W_bytes)
    W_md5_bytes = _hash.digest()
    W_md5_hex = _hash.hexdigest()

    print(W_md5_bytes)
    print(W_md5_hex)
    W_md5_int = int.from_bytes(W_md5_bytes, byteorder='big')

In [ ]:
def verify_signature(combined_message, pub_key, N):
    # Split the combined message into signature and message
    signature_bytes = combined_message[:128]
    message_bytes = combined_message[128:]

    # Convert signature bytes back to int
    signature = int.from_bytes(signature_bytes, 'big')

    # Calculate the hash of the message
    message_hash = int(hashlib.md5(message_bytes).hexdigest(), 16)

    # Decrypt the signature using the public key
    decrypted_signature = pow(signature, pub_key, N)

    # Compare the decrypted signature with the hash of the message
    if decrypted_signature == message_hash:
        print("Signature is valid.")
    else:
        print("Signature is not valid.")

In [ ]:
def verify_signature(_signature, _e, _n, message):
    # Decrypt the signature using the public key
    decrypted_signature = pow(signature, _e, N)

    # Hash the message
    mb = message.encode()
    h = MD5.new()
    h.update(mb)
    hmb = _hash.digest()

    # Compare the decrypted signature with the hash of the message
    if decrypted_signature == hmb:
        print("Signature is valid.")
    else:
        print("Signature is not valid.")

In [ ]:
#Deszyfrowanie - odbiorca
#Spojrzeć na podpis, odszyfrować podpis e modulo N i porównujemy zgodność tych wyników deszyfrowania i podpisu
decrypted = pow(signature, E, N)
#Liczymy hash z wiadomości i deszyfrujemy hash z wiadomości żeby sprawdzić, czy te dwie rzeczy się pokrywają

#odszyfrowany digested hash
print(decrypted)

In [ ]:
#Szyfrowanie podpisu 128 bitów i wiadomości w jednej częśći
#Przesunięcie bitowe i OR
message, signature = signed_message.split(':')
decrypted = pow(int(signature), E, N)
print(decrypted)